# David Beazley python talks

 This tutorial is a practical exploration of using Python coroutines (extended generators) for solving problems in data processing, event handling, and concurrent programming. The material starts off with generators and builds to writing a complete multitasking environment that can run thousands of concurrent tasks without using threads or using code based on event-driven callbacks (i.e., the "reactor" model). 

## 001. A Curious Course on Coroutines and Concurrency

Based on http://www.dabeaz.com/coroutines/

## 001.000 Assets

THE_FILE = "/tmp/access-log"


In [1]:
import sys
from pathlib import Path

In [2]:
from IPython.core.interactiveshell import InteractiveShell

THE_FILE = "/tmp/access-log"

InteractiveShell.ast_node_interactivity = "all"

### 001.001 Simple Generator

1. Make `countdown` a generator that loops through numbers lazily


In [3]:
# def countdown(n):
#     print("Counting down from", n)
#     print()
#     print("Done counting down")


# # Example use
# for i in countdown(10):
#     print(i, end=" ")

# solution


Counting down from 10
10 9 8 7 6 5 4 3 2 1 
Done counting down


### 001.002 A python tail -f

A generator that follows lines written to a real-time log file (like Unix 'tail -f'). To run this program, you need to have a log-file to work with. Run the program logsim.py to create a simulated web-server log (written in the file access-log). Leave this program running in the background for the next few parts.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. `follow` replicates tail -f on an open file
    1. jump to the end of the file
    1. Infinite loop
    1. Tries to get the next line, if not found it waits synchronously `sleep_for` seconds then try again; if not return the line

In [4]:
import time

sleep_for  = 0.1

def follow(thefile):
    1
    2 # while True:


# Example use
# logfile = open(THE_FILE)
# for line in follow(logfile):
#     print(line, end="")

# solution


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/access-log'

### 001.003 A simple pipeline

An example of using generators to set up a simple processing pipeline. Print all server log entries containing the word 'python'.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. `grep` is a generator that iterates through `lines` and if `pattern` is within a line, it will return it and wait
1. set up a pipeline
    1. open the file
    1. pass the file to `follow`, just reuse the earlier version, no need to redefine
    1. pass the output of `follow` to grep
1. Finally iterate through the pipeline and print the lines

In [ ]:
#

# def grep(pattern, lines):
#     1


# # Set up a processing pipe : tail -f | grep python
# 2.1
# 2.2
# the_pipeline = grep("python", loglines)

# Pull results out of the processing pipeline
3
#    print(f"==> {line}", end="")
# solution


### 001.004 Yield as an Expression

This function receives lines and prints out those that contain a substring.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. `grep` can now receive input - you can send `line` to it, instead of reading it from a file
1. set up a pipeline
    1. "prime" it, to make it ready to run
1.

In [ ]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        1
        if pattern in line:
            print(line)


g = grep("python")
2.1
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")

# solution


Looking for python
python generators rock!


### 001.005 Now with decorators, and catching close

A decorator function that eliminates the need to call .next() when starting a coroutine.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. Import what you need to create a decorator
1. create the `coroutine` decorator
    1. Use what you imported in 1
    1. extract the fn to a coroutine and prime it
    1. What returns what?
1. `grep` should be able to catch `close()`

In [ ]:
# from functools import 1


# def coroutine(func):
#     2.1
#     def start(*args, **kwargs):
#         2.2

#     2.3



# @coroutine
# def grep(pattern):
#     print("Looking for %s" % pattern)
#     while True:
#         line = yield
#         if pattern in line:
#             print(line, end=" ")
#     3
#         print()
#         print("Going away. Goodbye")

# g = grep("python")
# # Notice how you don't need a next() call here
# g.send("Yeah, but no, but yeah, but no")
# g.send("A series of tubes")
# g.send("python generators rock!")
# g.close()

# solution


Looking for python
python generators rock! 
Going away. Goodbye


### 001.006 Tail -f pipeline

Simple example of feeding data from a data source into a coroutine. This mirrors the 'tail -f' example from earlier.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. `follow` is similar to the previous example, but here we are sending the file to a coroutine, `target`
1. `printer` receives the line and prints it
1. `grep` is a filter
    1. it receives a line
    1. it sends it on, but reformatted as f"{pattern.upper()} > {line}" (you'll see why in the next example)
1. Set up the pipeline: `follow` is the producer, `printer` the consumer

In [ ]:
from functools import wraps
import time

def coroutine(func):
    @wraps(func)
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr

    return start

# A data source.  This is not a coroutine, but it sends
# data into one (target)
def follow(thefile, target):
    # 0 = an offset to add to...
    # 2 = ...the end of the file (0 is beginning, 1 is current position)
    thefile.seek(0, 2)
    while True:
        line = thefile.readline()
        if not line:
            time.sleep(0.1)  # Sleep briefly
            continue
        target.send(line)


# A filter.
# @coroutine
# def grep(pattern, target):
#     while True:
#         3.1
#         if pattern in line:
#             3.2 f"{pattern.upper()} > {line}"

# # A sink.  A coroutine that receives data
@coroutine
def printer():
    while True:
        line = yield
        print(line, end=" ")

# Using it
# f = open(THE_FILE)
# follow(4)

# solution


PYTHON > 198.49.180.40 - - [16/Sep/2023:01:20:31 -0600] "GET /python/tutorial/beazley_intro_python/Slides/SLIDE063.HTM HTTP/1.1" 200 984
 PYTHON > 24.15.187.198 - - [16/Sep/2023:01:20:31 -0600] "GET /python/tutorial/beazley_intro_python/Slides/SLIDE071.HTM HTTP/1.1" 200 1322
 PYTHON > 140.160.129.28 - - [16/Sep/2023:01:20:32 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE010.HTM HTTP/1.0" 200 1403
 PYTHON > 134.173.59.157 - - [16/Sep/2023:01:20:34 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE050.HTM HTTP/1.0" 200 989
 PYTHON > 84.110.148.125 - - [16/Sep/2023:01:20:37 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE008.HTM HTTP/1.0" 200 1231
 PYTHON > 67.207.145.238 - - [16/Sep/2023:01:20:38 -0600] "GET /python/tutorial/beazley_intro_python/Slides/SLIDE092.HTM HTTP/1.1" 200 1329
 

KeyboardInterrupt: 

### 001.007 Broadcast

An example of a coroutine broadcaster. This fans a data stream out to multiple targets.

0. Run `python katas/python_dabaez/solutions/001_logsym.py` on the CLI - it simulates an old school Apache server logs. You can't run it in a Jupter cell because it never ends
1. Reuses a lot of functions from earlier
1. broadcast passes the items to all the targets
1. Call `follow` and pipe into `broadcaster`, and have the latter send to different `grep / printer` combos with the 3 different strings "python", "ply", "swig"

In [ ]:
from functools import wraps
import time

def coroutine(func):
    @wraps(func)
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr

    return start

# A data source.  This is not a coroutine, but it sends
# data into one (target)
def follow(thefile, target):
    # 0 = an offset to add to...
    # 2 = ...the end of the file (0 is beginning, 1 is current position)
    thefile.seek(0, 2)
    while True:
        line = thefile.readline()
        if not line:
            time.sleep(0.1)  # Sleep briefly
            continue
        target.send(line)

# A filter.
@coroutine
def grep(pattern, target):
    while True:
        line = yield
        if pattern in line:
            target.send(f"{pattern.upper()} > {line}")

# A sink.  A coroutine that receives data
@coroutine
def printer():
    while True:
        line = yield
        print(line, end=" ")

# Broadcast a stream onto multiple targets
@coroutine
# def broadcast(targets):
#     while True:
#         item = yield
#         for 2:
#             ...

# Using it
# f = open(THE_FILE)
# follow(3)
# "python"
# "ply"
# "swig"

# solution


PYTHON > 24.125.38.188 - - [16/Sep/2023:01:24:41 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE117.HTM HTTP/1.0" 200 979
 PYTHON > 71.38.14.119 - - [16/Sep/2023:01:24:42 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE023.HTM HTTP/1.0" 200 1576
 PYTHON > 74.6.22.26 - - [16/Sep/2023:01:24:43 -0600] "GET /python/tutorial/beazley_intro_python/Slides/SLIDE084.HTM HTTP/1.0" 200 1474
 PYTHON > 134.67.6.11 - - [16/Sep/2023:01:24:45 -0600] "GET /python/tutorial/beazley_intro_python/Slides/SLIDE119.HTM HTTP/1.1" 304 -
 PLY > 129.97.51.195 - - [16/Sep/2023:01:24:46 -0600] "GET /ply/ply-1.7.tar.gz HTTP/1.1" 200 75085
 PYTHON > 203.73.43.189 - - [16/Sep/2023:01:24:48 -0600] "GET /python/tutorial/beazley_advanced_python/Slides/SLIDE056.HTM HTTP/1.0" 200 1504
 PLY > 71.183.55.2 - - [16/Sep/2023:01:24:48 -0600] "GET /ply/bookplug.gif HTTP/1.1" 304 -
 PLY > 80.91.229.6 - - [16/Sep/2023:01:24:49 -0600] "GET /ply/ply-1.4.tar.gz HTTP/1.1" 200 66002
 PYTHON > 193.252.14

KeyboardInterrupt: 